In [4]:
import pandas as pd
import numpy as np

data = pd.read_parquet('Data/pump_station_data.parquet')

print(data)
print(data.items())


                                  height  outflow  pump1_rpm  pump1_power  \
time                                                                        
2023-01-16 00:00:00.970000+00:00   167.0    533.0     1260.0        57.54   
2023-01-16 00:00:01.969000+00:00   167.0    534.0     1260.0        55.40   
2023-01-16 00:00:02.969000+00:00   167.0    527.0     1260.0        58.09   
2023-01-16 00:00:03.969000+00:00   167.0    521.0     1260.0        58.71   
2023-01-16 00:00:04.969000+00:00   167.0    521.0     1260.0        54.62   
...                                  ...      ...        ...          ...   
2023-03-01 23:59:55.342000+00:00   155.0    278.0        0.0         0.00   
2023-03-01 23:59:56.342000+00:00   155.0    278.0        0.0         0.00   
2023-03-01 23:59:57.343000+00:00   155.0    286.0        0.0         0.00   
2023-03-01 23:59:58.343000+00:00   155.0    292.0        0.0         0.00   
2023-03-01 23:59:59.342000+00:00   155.0    296.0        0.0         0.00   